In [9]:
import pandas as pd             
import numpy as np

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import make_scorer

from aux_fun import my_eval, my_grid_search_cv

import json

In [10]:
df = pd.read_csv('./data/train_imputed.csv')

In [11]:
y = df['NumberOfSales']

In [12]:
X = df.drop(df[['NumberOfSales','NumberOfCustomers', 'WindDirDegrees']], axis=1)

# Train-Test Split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
X_train = X_train.reset_index(drop =True)
y_train = y_train.reset_index(drop =True)

# Find the combination with best parameters

In [15]:
#parameters
params_array = [
    {'n_estimators' : 5},
    {'n_estimators' : 10},
    {'n_estimators' : 15},
    {'n_estimators' : 20},
    {'n_estimators' : 25},
    {'n_estimators' : 30},
    {'n_estimators' : 20,'max_features' : 1},
    {'n_estimators' : 20,'max_features' : 2},
    {'n_estimators' : 20,'max_features' : 5},
    {'n_estimators' : 20,'max_features' : 10},
    {'n_estimators' : 20,'max_features' : 20},
    {'n_estimators' : 20,'max_features' : 30},
    {'n_estimators' : 10,'max_depth':1},
    {'n_estimators' : 20,'max_depth':1},
    {'n_estimators' : 30,'max_depth':1},
    {'n_estimators' : 20,'max_depth':2},
    {'n_estimators' : 20,'max_depth':5},
    {'n_estimators' : 20,'max_depth':10},
    {'n_estimators' : 20,'max_depth':20},
    {'n_estimators' : 20,'max_depth':30},
    {'n_estimators' : 20,'max_depth':50},
    {'n_estimators' : 20,'min_samples_split':0.1},
    {'n_estimators' : 20,'min_samples_split':0.25},
    {'n_estimators' : 20,'min_samples_split':0.50},
    {'n_estimators' : 20,'min_impurity_split':0.20},
    {'n_estimators' : 20,'min_impurity_split':0.50},
    {'n_estimators' : 20,'min_impurity_split':0.80},
    {'n_estimators' : 15,'bootstrap':False},
    {'n_estimators' : 20,'bootstrap':False},
    {'n_estimators' : 25,'bootstrap':False},
]

In [16]:
results = my_grid_search_cv(RandomForestRegressor(), params_array, X_train, y_train, n_folds=5)

C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

...

C:\Users\luca\Anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\luca\Anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\luca\Anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\luca\Anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\luca\Anaconda3\lib\

C:\Users\luca\Anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\luca\Anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\luca\Anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\luca\Anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\luca\Anaconda3\lib\

C:\Users\luca\Anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\luca\Anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\luca\Anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\luca\Anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\luca\Anaconda3\lib\

C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [17]:
results

,Method,Folds,Parameters,Eval_test,R2,R2_month
0,"(DecisionTreeRegressor(criterion='mse', max_de...",5,{'n_estimators': 5},0.059781,0.889632,0.973421
1,"(DecisionTreeRegressor(criterion='mse', max_de...",5,{'n_estimators': 10},0.056463,0.901291,0.976260
2,"(DecisionTreeRegressor(criterion='mse', max_de...",5,{'n_estimators': 15},0.056056,0.904872,0.976806
3,"(DecisionTreeRegressor(criterion='mse', max_de...",5,{'n_estimators': 20},0.055632,0.906285,0.977029
4,"(DecisionTreeRegressor(criterion='mse', max_de...",5,{'n_estimators': 25},0.055080,0.907470,0.977409
5,"(DecisionTreeRegressor(criterion='mse', max_de...",5,{'n_estimators': 30},0.054987,0.908512,0.977552
6,"(DecisionTreeRegressor(criterion='mse', max_de...",5,"{'n_estimators': 20, 'max_features': 1}",0.227642,0.630389,0.600517
7,"(DecisionTreeRegressor(criterion='mse', max_de...",5,"{'n_estimators': 20, 'max_features': 2}",0.210860,0.672988,0.652500
8,"(DecisionTreeRegressor(criterion='mse', max_de...",5,"{'n_estimators': 20, 'max_features': 5}",0.154601,0.786896,0.798992
9,"(DecisionTreeRegressor(criterion='mse', max_de...",5,"{'n_estimators': 20, 'max_features': 10}",0.089771,0.883436,0.928051


In [18]:
results.to_csv('random_forest_results.csv')

# Model Building

In [19]:
best_params = results[results['Eval_test']==min(results['Eval_test'])]['Parameters'].reset_index(drop =True)[0]

In [20]:
best_params

{'n_estimators': 30}

In [21]:
rf = RandomForestRegressor(**best_params)
rf.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [22]:
y_pred = rf.predict(X_test)

In [23]:
print(y_pred)

[3653.83333333    0.         4300.23333333 ... 5319.5        5542.36666667
    0.        ]


In [24]:
print(y_test)

199480     3452
492279        0
500907     4679
63763      5232
74453         0
393492     4237
10444      3390
375668    11558
310103     2457
248408     8679
172477        0
481334     4816
141523     1766
57856      3305
19144      4121
442714     2509
198615     4005
102388     5892
201752     3889
249986     6864
385543        0
126690     2853
201562     5713
494046     5668
234057     4616
83925      7623
152551     4421
421235     4196
270066     5432
153996     3371
          ...  
183905     3828
67033      5458
356838     5935
37337         0
1022       4483
85432      4344
236582        0
12614      3003
323110     5531
317127     3944
514921     4167
310007        0
215660        0
406690     5104
91533      7221
303651        0
353483     4606
256305     4640
24320      3252
306740     3844
240634     6042
149711     3578
309371     4153
3414       4114
298003     5717
440202     5429
71752      6420
282435     5302
263394     5104
39            0
Name: NumberOfSales, Len

In [25]:
evaluation = my_eval(X_test, y_test, y_pred)
evaluation

C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\luca\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


0.04831883666667527

# Creation of the submission 

In [26]:
model_result={'method': 'Random Forest',
       'parameters':25,
       'evaluation': evaluation, 
       'r2':r2,
       'r2_grouped':r2grouped,
       'notes': 'None'
    }

NameError: name 'r2' is not defined

In [ ]:
with open('results.json', 'a') as fp:
    json.dump(model_result, fp, indent=2)
    fp.write('\n')